In [1]:
from __future__ import division, print_function, unicode_literals

In [2]:
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Tensorflow and Deep Learning

In this lab assignment, first you will learn how to build and train a neural network that recognises handwritten digits, and then you will build LeNet-5 CNN architecture, which is widely used for handwritten digit recognition. At the end of this lab assignment, you will make AlexNet CNN architecture, which won the 2012 ImageNet ILSVRC challenge.

---
# 1. Dataset
In the first part of the assignment, we use the MNIST dataset, which is a set of 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents. There are 70,000 images, and each image has 784 features. This is because each image is 28×28=784 pixels, and each feature simply represents one pixel's intensity, from 0 (white) to 255 (black). The following figure shows a few images from the MNIST dataset to give you a feel for the complexity of the classification task.

<img src="figs/1-mnist.png" style="width: 300px;"/>

To begin the assignment, first, use `mnist_data.read_data_sets` and download images and labels. It return two lists, called `mnist.test` with 10K images+labels, and `mnist.train` with 60K images+labels.

In [3]:
# TODO: Replace <FILL IN> with appropriate code

from tensorflow.examples.tutorials.mnist import input_data as mnist_data

mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


---
# 2. A One-Layer Neural Network
<img src="figs/2-comic1.png" style="width: 500px;"/>

Let's start by building a one-layer neural network. Handwritten digits in the MNIST dataset are 28x28 pixel greyscale images. The simplest approach for classifying them is to use the 28x28=784 pixels as inputs for a **one-layer neural network**. Each neuron in the network does a weighted sum of all of its inputs, adds a bias and then feeds the result through some non-linear activation function. Here we design a one-layer neural network with 10 output neurons since we want to classify digits into 10 classes (0 to 9).
<img src="figs/3-one_layer.png" style="width: 400px;"/>


For a classification problem, an *activation function* that works well is **softmax**. Applying softmax on a vector is done by taking the exponential of each element and then normalising the vector.
<img src="figs/4-softmax.png" style="width: 300px;"/>

We can summarise the behaviour of this single layer of neurons into a simple formula using a *matrix multiply*. If we give input data into the network in *mini-batch* of 100 images, it produces 100 predictions as the output. We define the **weights matrix $W$** with 10 columns, in which each column indicates the weight of a one class (a single digit), from 0 to 9. Using the first column of $W$, we can compute the weighted sum of all the pixels of the first image. This sum corresponds to the first neuron that points to the number 0. Using the second column of $W$, we do the same for the second neuron (number 1) and so on until the 10th neuron. We can then repeat the operation for the remaining 99 images in the mini-batch. If we call $X$ the matrix containing our 100 images (each row corresponds to one digit), all the weighted sums for our 10 neurons, computed on 100 images are simply $X.W$. Each neuron must now add its bias. Since we have 10 neurons, we have 10 bias constants. We finally apply the **softmax activation function** and obtain the formula describing a one-layer neural network, applied to 100 images.
<img src="figs/5-xw.png" style="width: 600px;"/>
<img src="figs/6-softmax2.png" style="width: 500px;"/>

Then, we need to use the **cross-entropy** to measure how good the predictions are, i.e., the distance between what the network tells us and what we know to be the truth. The cross-entropy is a function of weights, biases, pixels of the training image and its known label. If we compute the partial derivatives of the cross-entropy relatively to all the weights and all the biases, we obtain a **gradient**, computed for a given image, label and present value of weights and biases. We can update weights and biases by a fraction of the gradient and do the same thing again using the next batch of training images.
<img src="figs/7-cross_entropy.png" style="width: 600px;"/>

### Define Variables and Placeholders
First we define TensorFlow **variables** and **placeholders**. *Variables* are all the parameters that you want the training algorithm to determine for you (e.g., weights and biases). *Placeholders* are parameters that will be filled with actual data during training (e.g., training images). The shape of the tensor holding the training images is [None, 28, 28, 1] which stands for:
  - 28, 28, 1: our images are 28x28 (784) pixels x 1 value per pixel (grayscale). The last number would be 3 for color images and is not really necessary here.
  - None: this dimension will be the number of images in the mini-batch. It will be known at training time.

We also need an additional placeholder for the training labels that will be provided alongside training images.

In [4]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 1 layer of 10 softmax neurons
#
# · · · · · · · · · ·       (input data, flattened pixels)       X [batch, 784] 
# \x/x\x/x\x/x\x/x\x/    -- fully connected layer (softmax)      W [784, 10]     b[10]
#   · · · · · · · ·                                              Y_hat [batch, 10]

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))

# correct answers will go here
Y = tf.placeholder(tf.float32, shape=(None, 10))

# weights W[784, 10], 784 = 28 * 28
W = tf.Variable(tf.zeros([784, 10]))

# biases b[10]
b = tf.Variable(tf.zeros([10]))

### Build The Model
Now, we can make a **model** for a one-layer neural network. The formula is the one we explained before, i.e., $\hat{Y} = softmax(X . W + b)$. You can use the `tf.nn.softmax` and `tf.matmul` to build the model. Here, we need to use the `tf.reshape` to transform our 28x28 images into single vectors of 784 pixels.

In [5]:
# TODO: Replace <FILL IN> with appropriate code

# flatten the images into a single line of pixels
XX = tf.reshape(X, [-1, 784])

# The model
Y_hat = tf.nn.softmax(tf.matmul(XX, W) + b)

### Define The Cost Function
Now, we have model predictions $\hat{Y}$ and correct labels $Y$, so for each instance $i$ (image) we can compute the cross-entropy as the **cost function**: $cross\_entropy = -\sum(Y_i * log(\hat{Y}i))$. You can use `reduce_mean` to add all the components in a tensor.

In [6]:
# TODO: Replace <FILL IN> with appropriate code

cross_entropy = -Y * tf.log(Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy)

### Traine the Model
Now, select the gradient descent optimiser `GradientDescentOptimizer` and ask it to minimise the cross-entropy cost. In this step, TensorFlow computes the partial derivatives of the cost function relatively to all the weights and all the biases (the gradient). The gradient is then used to update the weights and biases. Set the learning rate is $0.005$.

In [7]:
# TODO: Replace <FILL IN> with appropriate code
learning_rate = 0.005

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

### Execute the Model
It is time to run the training loop. All the TensorFlow instructions up to this point have been preparing a computation graph in memory but nothing has been computed yet. The computation requires actual data to be fed into the placeholders. This is supplied in the form of a Python dictionary, where the keys are the names of the placeholders. During the trainig print out the cost every 200 steps. Moreove, after training the model, print out the accurray of the model by testing it on the test data.

In [8]:
# TODO: Replace <FILL IN> with appropriate code
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 5000

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

epoch 0: accuracy = 0.05999999865889549, loss = 0.230258509516716
epoch 200: accuracy = 0.5400000214576721, loss = 0.22087301313877106
epoch 400: accuracy = 0.699999988079071, loss = 0.21194878220558167
epoch 600: accuracy = 0.75, loss = 0.2005431205034256
epoch 800: accuracy = 0.7799999713897705, loss = 0.1942978799343109
epoch 1000: accuracy = 0.699999988079071, loss = 0.18901938199996948
epoch 1200: accuracy = 0.7599999904632568, loss = 0.17519883811473846
epoch 1400: accuracy = 0.800000011920929, loss = 0.1675708144903183
epoch 1600: accuracy = 0.75, loss = 0.16830676794052124
epoch 1800: accuracy = 0.75, loss = 0.1633121371269226
epoch 2000: accuracy = 0.8299999833106995, loss = 0.145156130194664
epoch 2200: accuracy = 0.7900000214576721, loss = 0.1477610021829605
epoch 2400: accuracy = 0.8299999833106995, loss = 0.14424888789653778
epoch 2600: accuracy = 0.7799999713897705, loss = 0.14051684737205505
epoch 2800: accuracy = 0.8600000143051147, loss = 0.13300196826457977
epoch 3000

---
# 2. Add More Layers

<img src="figs/8-comic2.png" style="width: 500px;"/>

Now, let's improve the recognition accuracy by adding more layers to the neural network. The neurons in the second layer, instead of computing weighted sums of pixels will compute weighted sums of neuron outputs from the previous layer. We keep the softmax function as the activation function on the last layer, but on intermediate layers we will use the the **sigmoid** activation function. So, let's build a five-layer fully connected neural network with the following structure, and train the model with the trainging data and print out its accuracy on the test data.
<img src="figs/9-five_layer.png" style="width: 500px;"/>

In [9]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with five layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1 [200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2 [100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3 [60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4 [30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5 [10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, tf.float32, [200]))
                 
W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [100]))

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [60]))

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [30]))

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784])

Y1_hat = tf.nn.sigmoid(tf.matmul(XX, W1) + B1)
Y2_hat = tf.nn.sigmoid(tf.matmul(Y1_hat, W2) + B2)
Y3_hat = tf.nn.sigmoid(tf.matmul(Y2_hat, W3) + B3)
Y4_hat = tf.nn.sigmoid(tf.matmul(Y3_hat, W4) + B4)
Y_hat = tf.nn.softmax(tf.matmul(Y4_hat, W5) + B5)

########################################
# define the cost function
########################################
cross_entropy = -Y * tf.log(Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy)
                
########################################
# define the optimizer
########################################
learning_rate = 0.005
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 5000
    
with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

epoch 0: accuracy = 0.12999999523162842, loss = 0.22968316078186035
epoch 200: accuracy = 0.07000000029802322, loss = 0.2353104054927826
epoch 400: accuracy = 0.07999999821186066, loss = 0.23792089521884918
epoch 600: accuracy = 0.18000000715255737, loss = 0.2314233034849167
epoch 800: accuracy = 0.07999999821186066, loss = 0.23475335538387299
epoch 1000: accuracy = 0.12999999523162842, loss = 0.23008239269256592
epoch 1200: accuracy = 0.11999999731779099, loss = 0.23202453553676605
epoch 1400: accuracy = 0.09000000357627869, loss = 0.2294391393661499
epoch 1600: accuracy = 0.07999999821186066, loss = 0.22868852317333221
epoch 1800: accuracy = 0.07999999821186066, loss = 0.23125390708446503
epoch 2000: accuracy = 0.09000000357627869, loss = 0.22978472709655762
epoch 2200: accuracy = 0.11999999731779099, loss = 0.23008231818675995
epoch 2400: accuracy = 0.15000000596046448, loss = 0.2301575243473053
epoch 2600: accuracy = 0.05000000074505806, loss = 0.23248250782489777
epoch 2800: accur

---
# 3. Special Care for Deep Networks
As layers were added, neural networks tended to converge with more difficulties. For example, the accuracy could stuck at 0.1. Here, we want to apply some updates to the network we built in the previous part to improve its performance. 

### ReLU Activation Function
<img src="figs/10-comic3.png" style="width: 500px;"/>
The sigmoid activation function is actually quite problematic in deep networks. It squashes all values between 0 and 1 and when you do so repeatedly, neuron outputs and their gradients can vanish entirely. An alternative activation function is **ReLU** that shows better performance compare to sigmoid. It looks like as below:
<img src="figs/11-relu.png" style="width: 300px;"/>

### A Better Optimizer
In very high dimensional spaces like here, **saddle points** are frequent. These are points that are not local minima, but where the gradient is nevertheless zero and the gradient descent optimizer stays stuck there. One possible solution to tackle this probelm is to use better optimizers, such as Adam optimizer `tf.train.AdamOptimizer`.

### Random Initialisations
When working with ReLUs, the best practice is to initialise bias values to small positive values, so that neurons operate in the non-zero range of the ReLU initially.

### Learning Rate
<img src="figs/12-comic4.png" style="width: 500px;"/>
With two, three or four intermediate layers, you can now get close to 98% accuracy, if you push the iterations to 5000 or beyond. But, the results are not very consistent, and the curves jump up and down by a whole percent. A good solution is to start fast and decay the learning rate exponentially from $0.005$ to $0.0001$ for example. In order to pass a different learning rate to the `AdamOptimizer` at each iteration, you will need to define a new placeholder and feed it a new value at each iteration through `feed_dict`. Here is the formula for exponential decay: $learning\_rate = lr\_min + (lr\_max - lr\_min) * e^{\frac{-i}{2000}}$, where $i$ is the iteration number.

### NaN?
In the network you built in the last section, you might see accuracy curve crashes and the console outputs NaN for the cross-entropy. It may happen, because you are attempting to compute a $log(0)$, which is indeed Not A Number (NaN). Remember that the cross-entropy involves a log, computed on the output of the softmax layer. Since softmax is essentially an exponential, which is never zero, we should be fine, but with 32 bit precision floating-point operations, exp(-100) is already a genuine zero. TensorFlow has a handy function that computes the softmax and the cross-entropy in a single step, implemented in a numerically stable way. To use it, you will need to separate the weighted sum plus bias on the last layer, before softmax is applied and then give it with the true values to the function `tf.nn.softmax_cross_entropy_with_logits`.

In the code below, apply the following changes and show their impact on the accuracy of the model on training data, as well as the test data:
* Replace the sigmoid activation function with ReLU
* Use the Adam optimizer
* Initialize weights with small random values between -0.2 and +0.2, and make sure biases are initialised with small positive values, for example 0.1
* Update the learning rate in different iterations. Start fast and decay the learning rate exponentially from $0.005$ to $0.0001$, i.e., 
```
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0
```
* Use `tf.nn.softmax_cross_entropy_with_logits` to prevent getting NaN in output.

In [10]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

import math

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))
step = tf.placeholder(tf.int32)

# variable learning rate
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000
learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate)*tf.math.exp(-1 * (step/decay_speed))

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
# when using RELUs, make sure biases are initialised with small positive values, for example 0.1
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, tf.float32, [200]))
                 
W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [100]))

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [60]))

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [30]))

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784])

Y1_hat = tf.nn.relu(tf.matmul(XX, W1) + B1)
Y2_hat = tf.nn.relu(tf.matmul(Y1_hat, W2) + B2)
Y3_hat = tf.nn.relu(tf.matmul(Y2_hat, W3) + B3)
Y4_hat = tf.nn.relu(tf.matmul(Y3_hat, W4) + B4)
logits_Y = tf.matmul(Y4_hat, W5) + B5
Y_hat = tf.nn.softmax(logits_Y)

########################################
# defining the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_Y, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 5000
    
with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y, step: i})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

epoch 0: accuracy = 0.15000000596046448, loss = 230.6082305908203
epoch 200: accuracy = 0.9399999976158142, loss = 13.915959358215332
epoch 400: accuracy = 0.8999999761581421, loss = 24.860937118530273
epoch 600: accuracy = 0.9599999785423279, loss = 9.695096969604492
epoch 800: accuracy = 0.9599999785423279, loss = 6.315128326416016
epoch 1000: accuracy = 0.9700000286102295, loss = 7.728232383728027
epoch 1200: accuracy = 0.9700000286102295, loss = 6.011181831359863
epoch 1400: accuracy = 0.9700000286102295, loss = 9.753211975097656
epoch 1600: accuracy = 0.9800000190734863, loss = 5.707936763763428
epoch 1800: accuracy = 0.9900000095367432, loss = 2.338412284851074
epoch 2000: accuracy = 0.9900000095367432, loss = 2.168837070465088
epoch 2200: accuracy = 1.0, loss = 0.2938225567340851
epoch 2400:

---
# 4. Overfitting and Dropout
<img src="figs/13-comic5.png" style="width: 500px;"/>
You will have noticed that cross-entropy curves for test and training data start disconnecting after a couple thousand iterations. The learning algorithm works on training data only and optimises the training cross-entropy accordingly. It never sees test data so it is not surprising that after a while its work no longer has an effect on the test cross-entropy which stops dropping and sometimes even bounces back up. 
<img src="figs/14-overfit.png" style="width: 500px;"/>
This disconnect is usually labeled **overfitting** and when you see it, you can try to apply a regularisation technique called **dropout**. In dropout, at each training iteration, you drop random neurons from the network. You choose a probability `pkeep` for a neuron to be kept, usually between 50% and 75%, and then at each iteration of the training loop, you randomly remove neurons with all their weights and biases. Different neurons will be dropped at each iteration. When testing the performance of your network of course you put all the neurons back (`pkeep = 1`).
<img src="figs/15-dropout.png" style="width: 500px;"/>
TensorFlow offers a dropout function to be used on the outputs of a layer of neurons. It randomly zeroes-out some of the outputs and boosts the remaining ones by `1 / pkeep`. You can add dropout after each intermediate layer in the network now. 

In the following code, use the dropout between each layer during the training, and set the probability `pkeep` once to $50%$ and another time to $75%$ and compare their results.

In [11]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))
step = tf.placeholder(tf.int32)

# variable learning rate
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000
learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate)*tf.math.exp(-1 * (step/decay_speed))

# probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
# when using RELUs, make sure biases are initialised with small positive values, for example 0.1
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, tf.float32, [200]))
                 
W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [100]))

W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [60]))

W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [30]))

W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 784])

Y1_hat = tf.nn.relu(tf.matmul(XX, W1) + B1)
Y1_hat_dropout = tf.nn.dropout(Y1_hat, pkeep)

Y2_hat = tf.nn.relu(tf.matmul(Y1_hat_dropout, W2) + B2)
Y2_hat_dropout = tf.nn.dropout(Y2_hat, pkeep)

Y3_hat = tf.nn.relu(tf.matmul(Y2_hat_dropout, W3) + B3)
Y3_hat_dropout = tf.nn.dropout(Y3_hat, pkeep)

Y4_hat = tf.nn.relu(tf.matmul(Y3_hat_dropout, W4) + B4)
Y4_hat_dropout = tf.nn.dropout(Y4_hat, pkeep)

logits_Y = tf.matmul(Y4_hat_dropout, W5) + B5
Y_hat = tf.nn.softmax(logits_Y)

########################################
# define the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_Y, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 5000

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y, pkeep:1.0})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y, step: i, pkeep:0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels, pkeep:1.0})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

epoch 0: accuracy = 0.12999999523162842, loss = 231.6296844482422
epoch 200: accuracy = 0.9200000166893005, loss = 30.307945251464844
epoch 400: accuracy = 0.949999988079071, loss = 16.5728759765625
epoch 600: accuracy = 0.9100000262260437, loss = 32.03618621826172
epoch 800: accuracy = 0.9599999785423279, loss = 9.600215911865234
epoch 1000: accuracy = 0.9800000190734863, loss = 9.155359268188477
epoch 1200: accuracy = 0.9700000286102295, loss = 9.609371185302734
epoch 1400: accuracy = 0.9399999976158142, loss = 23.525894165039062
epoch 1600: accuracy = 0.9300000071525574, loss = 27.019882202148438
epoch 1800: accuracy = 0.9700000286102295, loss = 5.1836838722229
epoch 2000: accuracy = 0.9800000190734863, loss = 6.076181888580322
epoch 2200: accuracy = 0.9700000286102295, loss = 9.861005783081055
epoch 2400: accuracy = 0.9800000190734863, loss = 4.623657703399658
epoch 2600: accuracy = 0.9900000095367432, loss = 3.561035633087158
epoch 2800: accuracy = 0.9900000095367432, loss = 2.114

---
# 6. Convolutional Network
<img src="figs/16-comic6.png" style="width: 500px;"/>
In the previous sections, all pixels of images flattened into a single vector, which was a really bad idea. Handwritten digits are made of shapes and we discarded the shape information when we flattened the pixels. However, we can use **convolutional neural networks (CNN)** to take advantage of shape information. CNNs apply *a series of filters* to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains three components:
  - **Convolutional layers**: apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. Convolutional layers then typically apply a ReLU activation function to the output to introduce nonlinearities into the model.
  - **Pooling layers**: downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values.
  - **Dense (fully connected) layers**: perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.
  
Typically, a CNN is composed of a *stack of **convolutional modules*** that perform feature extraction. Each *module* consists of a *convolutional layer* followed by a *pooling layer*. The last convolutional module is followed by one or more dense layers that perform classification. The final dense layer in a CNN contains a single neuron for each target class in the model, with a softmax activation function to generate a value between 0-1 for each neuron. We can interpret the softmax values for a given image as relative measurements of how likely it is that the image falls into each target class.

Now, let us build a convolutional network for handwritten digit recognition. In this assignment, we will use the architecture shown in the following figure that has three convolutional layers, one fully-connected layer, and one softmax layer. Notice that the second and third convolutional layers have a stride of two that explains why they bring the number of output values down from 28x28 to 14x14 and then 7x7. A convolutional layer requires a weights tensor like `[4, 4, 3, 2]`, in which the first two numbers define the size of a filter (map), the third number shows the *depth* of the filter that is the number of *input channel*, and the last number shows the number of *output channel*. The output channel defines the number of times that we repeat the same thing with a different set of weights in one layer. In our implementation, we assume the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected layer is 200.
<img src="figs/17-arch1.png" style="width: 600px;"/>

Convolutional layers can be implemented in TensorFlow using the `tf.nn.conv2d` function, which performs the scanning of the input image in both directions using the supplied weights. This is only the weighted sum part of the neuron. You still need to add a bias and feed the result through an activation function. The padding strategy that works here is to copy pixels from the sides of the image. All digits are on a uniform background so this just extends the background and should not add any unwanted shapes.

In [12]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·      (input data, 1-deep)               X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @   -- conv. layer 5x5x1=>4 stride 1      W1 [5, 5, 1, 4]        B1 [4]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 4]
#   @ @ @ @ @ @ @ @     -- conv. layer 5x5x4=>8 stride 2      W2 [5, 5, 4, 8]        B2 [8]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 8]
#     @ @ @ @ @ @       -- conv. layer 4x4x8=>12 stride 2     W3 [4, 4, 8, 12]       B3 [12]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 12] => reshaped to YY [batch, 7*7*12]
#      \x/x\x\x/        -- fully connected layer (relu)       W4 [7*7*12, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/         -- fully connected layer (softmax)    W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))
step = tf.placeholder(tf.int32)

# variable learning rate
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000
learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate)*tf.math.exp(-1 * (step/decay_speed))

# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected
# layer is 200
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, tf.float32, [4]))
                 
W2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [8]))

W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [12]))

W4 = tf.Variable(tf.truncated_normal([7 * 7 * 12, 200], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [200]))

W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, shape=[-1, 7 * 7 * 12])
Y4_hat = tf.nn.relu(tf.matmul(YY_hat, W4) + B4)
logits_Y = tf.matmul(Y4_hat, W5) + B5
Y_hat = tf.nn.softmax(logits_Y)

########################################
# define the cost function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_Y, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 2000

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y, step: i})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print('test data: accuracy = {}, loss = {}'.format(a, c))

epoch 0: accuracy = 0.10999999940395355, loss = 238.54312133789062
epoch 200: accuracy = 0.9800000190734863, loss = 7.833214282989502
epoch 400: accuracy = 0.9599999785423279, loss = 8.717127799987793
epoch 600: accuracy = 0.9599999785423279, loss = 6.053647994995117
epoch 800: accuracy = 0.9900000095367432, loss = 4.473823070526123
epoch 1000: accuracy = 1.0, loss = 0.9588204622268677
epoch 1200: accuracy = 1.0, loss = 0.8714559674263
epoch 1400: accuracy = 1.0, loss = 1.180545687675476
epoch 1600: accuracy = 1.0, loss = 1.7049915790557861
epoch 1800: accuracy = 0.9900000095367432, loss = 3.325397253036499
test data: accuracy = 0.9871000051498413, loss = 4.374074459075928


# 7. Improve The Performance
A good approach to sizing your neural networks is to implement a network that is a little too constrained, then give it a bit more degrees of freedom and add dropout to make sure it is not overfitting. This ends up with a fairly optimal network for your problem. In the above model, we set the output channel to 4 in the first convolutional layer, which means that we repeat the same filter shape (but with different weights) four times. If we assume that those filters evolve during training into shape recognisers, you can intuitively see that this might not be enough for our problem. Handwritten digits are made from more than 4 elemental shapes. So let us bump up the filter sizes a little, and also increase the number of filters in our convolutional layers from 4, 8, 12 to 6, 12, 24 and then add dropout on the fully-connected layer. The following figure shows the new architecture you should build. Please complete the following code based on the given architecture and dropout technique.
<img src="figs/18-arch2.png" style="width: 600px;"/>

In [13]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·    (input data, 1-deep)                 X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @ -- conv. layer 6x6x1=>6 stride 1        W1 [5, 5, 1, 6]        B1 [6]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 6]
#   @ @ @ @ @ @ @ @   -- conv. layer 5x5x6=>12 stride 2       W2 [5, 5, 6, 12]        B2 [12]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 12]
#     @ @ @ @ @ @     -- conv. layer 4x4x12=>24 stride 2      W3 [4, 4, 12, 24]       B3 [24]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 24] => reshaped to YY [batch, 7*7*24]
#      \x/x\x\x/ ✞    -- fully connected layer (relu+dropout) W4 [7*7*24, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/       -- fully connected layer (softmax)      W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))
step = tf.placeholder(tf.int32)

# variable learning rate
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000
learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate)*tf.math.exp(-1 * (step/decay_speed))

# probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 6, 12, 24, and the size of fully connected
# layer is 200
W1 = tf.Variable(tf.truncated_normal([6, 6, 1, 6], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, tf.float32, [6]))
                 
W2 = tf.Variable(tf.truncated_normal([5, 5, 6, 12], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [12]))

W3 = tf.Variable(tf.truncated_normal([4, 4, 12, 24], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [24]))

W4 = tf.Variable(tf.truncated_normal([7 * 7 * 24, 200], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [200]))

W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, shape=[-1, 7 * 7 * 24])
Y4_hat = tf.nn.relu(tf.matmul(YY_hat, W4) + B4)

YY4_hat = tf.nn.dropout(Y4_hat, pkeep)    
                        
logits_Y = tf.matmul(YY4_hat, W5) + B5
Y_hat = tf.nn.softmax(logits_Y)

########################################
# define the Loss function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_Y, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)
                        
########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 3000

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y, pkeep:1.0})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y, step: i, pkeep:0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels, pkeep:1.0})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

epoch 0: accuracy = 0.05999999865889549, loss = 275.8196716308594
epoch 200: accuracy = 0.9700000286102295, loss = 12.020134925842285
epoch 400: accuracy = 0.9599999785423279, loss = 13.418631553649902
epoch 600: accuracy = 0.9900000095367432, loss = 2.401998519897461
epoch 800: accuracy = 1.0, loss = 0.5276710391044617
epoch 1000: accuracy = 0.9900000095367432, loss = 2.600630044937134
epoch 1200: accuracy = 0.9800000190734863, loss = 5.633187294006348
epoch 1400: accuracy = 1.0, loss = 0.12601058185100555
epoch 1600: accuracy = 0.9900000095367432, loss = 2.670727014541626
epoch 1800: accuracy = 0.9900000095367432, loss = 9.085548400878906
epoch 2000: accuracy = 0.9900000095367432, loss = 1.9051796197891235
epoch 2200: accuracy = 1.0, loss = 0.29427003860473633
epoch 2400: accuracy = 0.9700000286102295, loss = 4.961801528930664
epoch 2600: accuracy = 1.0, loss = 0.060456402599811554
epoch 2800: accuracy = 1.0, loss = 0.5871527791023254
test data: accuracy = 0.9911999702453613, loss = 

---
# 8. Tensorflow Layers Module
The TensorFlow **layers** `tf.layers` module provides a high-level API that makes it easy to construct a neural network. It provides methods that facilitate: (i) the creation of dense (fully connected) layers and convolutional layers, (ii) adding activation functions, and (iii) applying dropout regularization. In this section use the module `tf.layers` to build the network you made in section 7.

In [14]:
# TODO: Replace <FILL IN> with appropriate code

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
Y = tf.placeholder(tf.float32, shape=(None, 10))
step = tf.placeholder(tf.int32)

# variable learning rate
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000
learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate)*tf.math.exp(-1 * (step/decay_speed))

# probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

########################################
# build the model
########################################
conv1 = tf.layers.conv2d(inputs=X, filters=6, kernel_size=[6, 6], padding="same", activation=tf.nn.relu)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

conv2 = tf.layers.conv2d(inputs=conv1, filters=12, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

conv3 = tf.layers.conv2d(inputs=pool2, filters=24, kernel_size=[4, 4], padding="same", activation=tf.nn.relu)

pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

pool3_flat = tf.reshape(pool3, [-1, 7 * 7 * 24])

dense = tf.layers.dense(inputs=pool3_flat, units=200, activation=tf.nn.relu)

# Add dropout operation
dropout = tf.layers.dropout(inputs=dense, rate=pkeep)

# Logits layer
logits_Y = tf.layers.dense(inputs=dropout, units=10)
Y_hat = tf.nn.softmax(logits_Y)

########################################
# define the Loss function
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_Y, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)
                        
########################################
# execute the model
########################################
correct_prediction = tf.equal(tf.argmax(Y_hat, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# init
init = tf.global_variables_initializer()

batch_size = 100
n_epochs = 3000
    
with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 200 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, Y: batch_y, pkeep:1.0})
            print('epoch {}: accuracy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, Y: batch_y, step: i, pkeep:0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y: mnist.test.labels, pkeep:1.0})
    print('test data: accuracy = {}, loss = {}'.format(a, c))  

epoch 0: accuracy = 0.07000000029802322, loss = 231.43771362304688
epoch 200: accuracy = 0.9599999785423279, loss = 18.3535213470459
epoch 400: accuracy = 0.9599999785423279, loss = 12.006437301635742
epoch 600: accuracy = 0.9800000190734863, loss = 5.258813381195068
epoch 800: accuracy = 0.9900000095367432, loss = 1.1983654499053955
epoch 1000: accuracy = 1.0, loss = 2.1906402111053467
epoch 1200: accuracy = 0.9700000286102295, loss = 6.606575965881348
epoch 1400: accuracy = 1.0, loss = 0.2635703682899475
epoch 1600: accuracy = 1.0, loss = 0.17613378167152405
epoch 1800: accuracy = 0.9900000095367432, loss = 1.6677790880203247
epoch 2000: accuracy = 1.0, loss = 0.654496431350708
epoch 2200: accuracy = 0.9900000095367432, loss = 2.6451947689056396
epoch 2400: accuracy = 0.9900000095367432, loss = 1.9448124170303345
epoch 2600: accuracy = 1.0, loss = 0.44833850860595703
epoch 2800: accuracy = 1.0, loss = 0.007541372906416655
test data: accuracy = 0.9929999709129333, loss = 2.14449286460

---
# 9. Keras
Keras is a high-level API to build and train deep learning models. It's used for fast prototyping, advanced research, and production. `tf.keras` is TensorFlow's implementation of the Keras API specification. To work with Keras, you need to import `tf.keras` as part of your TensorFlow program setup.
```
import tensorflow as tf
from tensorflow.keras import layers
```
#### Build a model
In Keras, you assemble **layers** to build a model, i.e., a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model. For example, the following code builds a simple, fully-connected network (i.e., multi-layer perceptron):
```
model = tf.keras.Sequential()
# adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# add another
model.add(layers.Dense(64, activation='relu'))
# add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))
```
There are many `tf.keras.layers` available with some common constructor parameters:
* `activation`: set the activation function for the layer, which is specified by the name of a built-in function or as a callable object.
* `kernel_initializer` and `bias_initializer`: the initialization schemes that create the layer's weights (weight and bias).
* `kernel_regularizer` and `bias_regularizer`: the regularization schemes that apply the layer's weights (weight and bias), such as L1 or L2 regularization.

#### Train and evaluate
After you construct a model, you can configure its learning process by calling the `compile` method:
```
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```
The method `tf.keras.Model.compile` takes three important arguments:
* `optimizer`: it specifies the training procedure, e.g., `tf.train.AdamOptimizer` and `tf.train.GradientDescentOptimizer`.
* `loss`: the cost function to minimize during optimization, e.g., mean square error (mse), categorical_crossentropy, and binary_crossentropy.
* `metrics`: used to monitor training, e.g., `accuracy`.

The next step after confiuring the model is to train it by calling the `model.fit` method and giving it training data as its input. After training the model you can call `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods to evaluate the inference-mode loss and metrics for the data provided or predict the output of the last layer in inference for the data provided, respectively.

You can read more about Keras [here](https://www.tensorflow.org/guide/keras).

In this task, please use Keras to rebuild the network you made in section 7.

In [3]:
# TODO: Replace <FILL IN> with appropriate code

from tensorflow import keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import utils as np_utils


# to reset the Tensorflow default graph
reset_graph()

n_epochs = 10
batch_size = 100

########################################
# load the mnist data
########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Rscaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

########################################
# buid the model
########################################
model = Sequential()

model.add(Conv2D(6, kernel_size=[6, 6], activation='relu', input_shape=[28, 28, 1]))

model.add(MaxPooling2D(pool_size=[2, 2], strides=1))

model.add(Conv2D(12, kernel_size=[5, 5], activation='relu'))

model.add(MaxPooling2D(pool_size=[2, 2], strides=2))

model.add(Conv2D(24, kernel_size=[4, 4], activation='relu'))

model.add(MaxPooling2D(pool_size=[2, 2], strides=2))

model.add(Flatten())

model.add(Dense(200, activation='relu'))

model.add(Dropout(0.25))

model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.2847 - acc: 0.9112 - val_loss: 0.0814 - val_acc: 0.9745
Epoch 2/10
60000/60000 [==============================] - 4s 62us/step - loss: 0.0826 - acc: 0.9749 - val_loss: 0.0493 - val_acc: 0.9836
Epoch 3/10
60000/60000 [==============================] - 6s 92us/step - loss: 0.0605 - acc: 0.9804 - val_loss: 0.0402 - val_acc: 0.9869
Epoch 4/10
60000/60000 [==============================] - 10s 166us/step - loss: 0.0480 - acc: 0.9852 - val_loss: 0.0341 - val_acc: 0.9884
Epoch 5/10
60000/60000 [==============================] - 10s 169us/step - loss: 0.0429 - acc: 0.9870 - val_loss: 0.0351 - val_acc: 0.9886
Epoch 6/10
60000/60000 [==============================] - 10s 163us/step - loss: 0.0362 - acc: 0.9887 - val_loss: 0.0288 - val_acc: 0.9896
Epoch 7/10
60000/60000 [==============================] - 9s 157us/step - loss: 0.0335 - acc: 0.9894 - val_loss: 0.0312 -

---
# 10. Implement LeNet-5
In this section, you should implement **LeNet-5** either using Tensorflow or Keras. Please take a look at its [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) before starting to implement it.
The LeNet-5 architecture is perhaps the most widely known CNN architecture. It was created by Yann LeCun in 1998 and widely used for handwritten digit recognition (MNIST). It is composed of the layers shown in the following table.
<img src="figs/19-letnet5.png" style="width: 600px;"/>
There are a few extra details to be noted:
* MNIST images are 28×28 pixels, but they are zero-padded to 32×32 pixels and normalized before being fed to the network. The rest of the network does not use any padding, which is why the size keeps shrinking as the image progresses through the network.
* The average pooling layers are slightly more complex than usual: each neuron computes the mean of its inputs, then multiplies the result by a learnable coefficient and adds a learnable bias term, then finally applies the activation function.
* Most neurons in layer C3 maps are connected to neurons in only three or four S2 maps (instead of all six S2 maps). See table 1 in the [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) for details.
* The output layer is a bit special: instead of computing the dot product of the inputs and the weight vector, each neuron outputs the square of the Euclidian distance between its input vector and its weight vector. Each output measures how much the image belongs to a particular digit class. The cross-entropy cost function is now preferred, as it penalizes bad predictions much more, producing larger gradients and thus converging faster.

In [4]:
# TODO: Build the LetNet-5 model, and test it on MNIST

from tensorflow import keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras import utils as np_utils


# to reset the Tensorflow default graph
reset_graph()

n_epochs = 10
batch_size = 100

########################################
# load the mnist data
########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


# Rscaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train /= 255
x_test /= 255

#Padding the images by 2 : Extra Task 1
X_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

print(X_train.shape[1:])

#Normalization
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
X_train = (X_train - mean_px)/(std_px)
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)


########################################
# buid the model
########################################
model = Sequential()

model.add(Conv2D(6, kernel_size=[5, 5], activation='tanh', input_shape=[28, 28, 1], strides=1, padding="same"))

model.add(AveragePooling2D(pool_size=[2, 2], strides=2, padding='valid'))

model.add(Conv2D(16, kernel_size=[5, 5], activation='tanh', strides=1, padding="valid"))

model.add(AveragePooling2D(pool_size=[2, 2], strides=2, padding='valid'))

model.add(Conv2D(120, kernel_size=[5, 5], activation='tanh', strides=1, padding="valid"))

model.add(Flatten())

model.add(Dense(84, activation='tanh'))

model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(32, 32, 1)
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 178us/step - loss: 0.0146 - acc: 0.9955 - val_loss: 0.0329 - val_acc: 0.9902
Epoch 2/10
60000/60000 [==============================] - 9s 148us/step - loss: 0.0138 - acc: 0.9955 - val_loss: 0.0379 - val_acc: 0.9898
Epoch 3/10
60000/60000 [==============================] - 10s 164us/step - loss: 0.0124 - acc: 0.9960 - val_loss: 0.0327 - val_acc: 0.9898
Epoch 4/10
60000/60000 [==============================] - 10s 164us/step - loss: 0.0119 - acc: 0.9959 - val_loss: 0.0325 - val_acc: 0.9909
Epoch 5/10
60000/60000 [==============================] - 10s 163us/step - loss: 0.0105 - acc: 0.9966 - val_loss: 0.0331 - val_acc: 0.9894
Epoch 6/10
60000/60000 [==============================] - 10s 162us/step - loss: 0.0099 - acc: 0.9968 - val_loss: 0.0345 - val_acc: 0.9906
Epoch 7/10
60000/60000 [==============================] - 10s 160us/step - loss: 0.0099 - acc: 0.9967 - v

In [7]:
# Extra Changes

from tensorflow import keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras import utils as np_utils

# to reset the Tensorflow default graph
reset_graph()

n_epochs = 10
batch_size = 100

########################################
# load the mnist data
########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


# Rscaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train /= 255
x_test /= 255

#Padding the images by 2 : Extra Task 1
X_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

print(X_train.shape[1:])

#Normalization
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
X_train = (X_train - mean_px)/(std_px)

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)


########################################
# buid the model
########################################
model = Sequential()

model.add(Conv2D(6, kernel_size=[5, 5], activation='tanh', input_shape=[28, 28, 1], strides=1, padding="same"))

##Extra Task 2 : Custom AveragePoolingLayer
model.add(AveragePooling2D(pool_size=[2, 2], strides=2, padding='valid'))
model.add(AveragePoolingLayer(output_dimension = (14, 14, 6)))

#Extra Task 3: Not Sure This is correct solution. Even concatination can be used
model.add(Dropout(0.30))
model.add(Conv2D(16, kernel_size=[5, 5], activation='tanh', strides=1, padding="valid"))

##Extra Task 2 : Custom AveragePoolingLayer
model.add(AveragePooling2D(pool_size=[2, 2], strides=2, padding='valid'))
model.add(AveragePoolingLayer(output_dimension = (5, 5, 16)))

model.add(Conv2D(120, kernel_size=[5, 5], activation='tanh', strides=1, padding="valid"))

model.add(Flatten())

model.add(Dense(84, activation='tanh'))

model.add(Dense(10, activation='softmax'))
#model.add(OutputLayer(10))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(32, 32, 1)
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.7930 - acc: 0.7266 - val_loss: 0.2328 - val_acc: 0.9335
Epoch 2/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.2319 - acc: 0.9292 - val_loss: 0.1406 - val_acc: 0.9578
Epoch 3/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.1706 - acc: 0.9474 - val_loss: 0.1079 - val_acc: 0.9668
Epoch 4/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.1375 - acc: 0.9570 - val_loss: 0.0863 - val_acc: 0.9723
Epoch 5/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.1191 - acc: 0.9619 - val_loss: 0.0787 - val_acc: 0.9750
Epoch 6/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.1035 - acc: 0.9689 - val_loss: 0.0682 - val_acc: 0.9781
Epoch 7/10
60000/60000 [==============================] - 4s 63us/step - loss: 0.0935 - acc: 0.9712 - val_loss: 0.0

In [6]:
##Extra Task 2 : Custom AveragePoolingLayer
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class AveragePoolingLayer(Layer):
    def __init__(self, output_dimension, **kwargs):
        super(AveragePoolingLayer, self).__init__(**kwargs)
        self.output_dimension = output_dimension

    def build(self, input_shape):
        self.weights_new = self.add_weight(name='weights2',
                              shape=(
                               int(self.output_dimension[0]), 
                               int(self.output_dimension[1]), 
                               int(self.output_dimension[2])),
                               initializer='uniform',
                               trainable=True)
        self.bias_new  = self.add_weight(name='bias',
                                 shape=(
                               int(self.output_dimension[0]), 
                               int(self.output_dimension[1]), 
                               int(self.output_dimension[2])),
                               initializer='uniform',
                               trainable=True)
        super(AveragePoolingLayer, self).build(input_shape) 

    def call(self, inputs):
        return K.tanh((inputs * self.weights_new + self.bias_new)) 

    def compute_output_shape(self, input_shape):
        return (input_shape)

In [19]:
#Task 4 : Custom Output Layer
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class OutputLayer(Layer):
    def __init__(self, dim, **kwargs):
        super(OutputLayer, self).__init__(**kwargs)
        self.dim = dim
  
    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.dim),
                                  initializer='uniform',
                                  trainable=True)
        super(OutputLayer, self).build(input_shape)

    def call(self, inputs):
        l2 = K.sum(K.pow(inputs,2), axis=1)
        return l2

    def compute_output_shape(self, input_shape):
        return (self.dim)

---
# 11. Implement AlexNet
In the last section, you should implement **AlexNet** either using Tensorflow or Keras. Again, please take a look at its [paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) before start to implement it.
The AlexNet CNN architecture won the [ImageNet ILSVRC challenge](http://www.image-net.org/challenges/LSVRC/2012/) in 2012 by a large margin. It was developed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It is quite similar to LeNet-5, only much larger and deeper, and it was the first to stack convolutional layers directly on top of each other, instead of stacking a pooling layer on top of each convolutional layer. The following table presents this architecture.
<img src="figs/20-alexnet.png" style="width: 600px;"/>
To train the model, we need a big dataset, however, in this assignment you are going to to assign the pretrained weights to your model, using `tf.Variable.assign`. You can download the pretrained weights from [bvlc_alexnet.npy](https://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy). This file is a NumPy array file created by the python. After you read this file, you will receive a python dictionary with a <key, value> pair for each layer. Each key is one of the layers names, e.g., `conv1`, and each value is a list of two values: (1) weights, and (2) biases of that layer. Part of the function to load the weights and biases to your model is given, and you need to complete it.

Here is what you see if you read and print the shape of each layer from the file:
```
weight_dic = np.load("bvlc_alexnet.npy", encoding="bytes").item()
for layer in weights_dic:
    print("-" * 20)
    print(layer)
    for wb in weights_dic[layer]:
        print(wb.shape)

#--------------------
# fc8
# (4096, 1000) # weights
# (1000,) # bias
#--------------------
# fc7
# (4096, 4096) # weights
# (4096,) # bias
#--------------------
# fc6
# (9216, 4096) # weights
# (4096,) # bias
#--------------------
# conv5
# (3, 3, 192, 256) # weights
# (256,) # bias
#--------------------
# conv4
# (3, 3, 192, 384) # weights
# (384,) # bias
#--------------------
# conv3
# (3, 3, 256, 384) # weights
# (384,) # bias
#--------------------
# conv2
# (5, 5, 48, 256) # weights
# (256,) # bias
#--------------------
# conv1
# (11, 11, 3, 96) # weights
# (96,) # bias
```


In [7]:
import tensorflow as tf
import numpy as np
from tensorflow import keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D,MaxPooling2D
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend as K
from numpy import array

# to reset the Tensorflow default graph
reset_graph()

setattr(tf.contrib.rnn.GRUCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.BasicLSTMCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.MultiRNNCell, '__deepcopy__', lambda self, _: self)

def create_model():
 
    weights_dic = np.load('bvlc_alexnet.npy', encoding='bytes').item()

    model = Sequential()

    model.add(Conv2D(96, kernel_size=[11, 11], activation='relu', input_shape=[224, 224, 3], strides=4, padding="same", name="conv1"))

    model.add(MaxPooling2D(pool_size=[2, 2], strides=2, padding="valid"))

    model.add(Conv2D(256, kernel_size=[5, 5], activation='relu', strides=1, padding="same", name="conv2"))

    model.add(MaxPooling2D(pool_size=[3, 3], strides=2, padding="valid"))

    model.add(Conv2D(384, kernel_size=[3, 3], activation='relu', strides=1, padding="same", name="conv3"))

    model.add(Conv2D(384, kernel_size=[3, 3], activation='relu', strides=1, padding="same", name="conv4"))

    model.add(Conv2D(256, kernel_size=[3, 3], activation='relu', strides=1, padding="same", name="conv5"))

    model.add(MaxPooling2D(pool_size=[3, 3], strides=2, padding="valid"))

    model.add(Flatten())
 
    model.add(Dense(4096, activation='relu', name="dens1"))
 
    ##model.add(Dropout(np.var(1.0)))
    
    model.add(Dense(4096, activation='relu', name="dens2"))
    
    ##model.add(Dropout(np.var(1.0)))
    
    model.add(Dense(1000, activation='softmax', name="dens3"))

    c2 = np.concatenate((weights_dic["conv2"][0],weights_dic["conv2"][0]),axis=2);
    c4 = np.concatenate((weights_dic["conv4"][0],weights_dic["conv4"][0]),axis=2);
    c5 = np.concatenate((weights_dic["conv5"][0],weights_dic["conv5"][0]),axis=2);

    model.layers[0].set_weights([weights_dic["conv1"][0], weights_dic["conv1"][1]])
    model.layers[2].set_weights([c2, weights_dic["conv2"][1]])
    model.layers[4].set_weights([weights_dic["conv3"][0], weights_dic["conv3"][1]])
    model.layers[5].set_weights([c4, weights_dic["conv4"][1]])
    model.layers[6].set_weights([c5, weights_dic["conv5"][1]])
    model.layers[9].set_weights([weights_dic["fc6"][0], weights_dic["fc6"][1]]) 
    model.layers[10].set_weights([weights_dic["fc7"][0], weights_dic["fc7"][1]]) #11
    model.layers[11].set_weights([weights_dic["fc8"][0], weights_dic["fc8"][1]]) #13
    return model

def train():
    model = create_model()
    model.save('final_m12.hdf5')

In [8]:
train()

#### Test the model
After building the AlexNet model, you can test it on different images and present the accuracy of the model. To do so, first you need to use **OpenCV** library to make the images ready to give as input to the model. OpenCV is a library used for image processing. Below you can see how to read an image file and pre-process it using OpenCV to give it to the model. However, you need to complete the code and test the accuracy of your model. The teset images (shown below) are available in the `test_images` folder.
<table width="100%">
<tr>
<td><img src="test_images/test_image1.jpg" style="width:200px;"></td>
<td><p align="center"><img src="test_images/test_image2.jpg" style="width:200px;"></td>
<td align="right"><img src="test_images/test_image3.jpg" style="width:200px;"></td>
</tr>

In [9]:
# TODO: Replace <FILL IN> with appropriate code
# test the AlexNet model on the given images

# TODO: Replace <FILL IN> with appropriate code
# test the AlexNet model on the given images

import os
import urllib.request
import argparse
import sys
import cv2
import tensorflow as tf
import numpy as np
import caffe_classes
import glob
from tensorflow.keras.models import Sequential

def softmax(z): return np.exp(z)/((np.exp(z)).sum())
#get list of all images
current_dir = os.getcwd()
image_path = os.path.join(current_dir, 'test_images')
img_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith('.jpg')]

#load all images
imgs = []
for f in img_files:
    imgs.append(cv2.imread(f))

model = create_model()    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.load_weights("final_m2.hdf5") #Load the model
    # loop over all images
    for i, image in enumerate(imgs):
        # convert image to float32 and resize to (227x227)
        img = cv2.resize(image.astype(np.float32), (224, 224))
        
        # subtract the ImageNet mean
        # Mean subtraction per channel was used to center the data around zero mean for each channel (R, G, B).
        # This typically helps the network to learn faster since gradients act uniformly for each channel.
        imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)
        img -= imagenet_mean
        
        # reshape as needed to feed into model
        img = img.reshape((1, 224, 224, 3))
        
        p = model.predict_classes(img)
        #Not Sure About This File: May be it doesn't match index
        res = caffe_classes.class_names[p[0]] #find the max probility
        print(res)

water bottle
sea slug, nudibranch
panpipe, pandean pipe, syrinx
